In [1]:
import os
import re
import json
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
from scipy import ndimage
from scipy import optimize 
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import utils

def scatter(w, k):
    return k * (1 / w**4)

def r_squared(y, y_hat):
    residuals = y - y_hat
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y-np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared


root_dir_walk = os.walk('..')

all_traces_summaries = []


for dirpath, dirnames, filenames in root_dir_walk:
    if (e := re.search('\.\./([0-9]+)', dirpath)) is not None:
        experiment_number = int(e.group(1))
        data = {}
        if 1 <= experiment_number <= 22:
            #if experiment_number not in experiment_configs.keys():
            for filename in filenames:
                fpath = os.path.join(dirpath, filename)
                if re.search('experiment-[0-9]+-per-well-summary\.csv', filename) is not None:
                    all_traces_summaries.append(os.path.join(dirpath, filename))

df  = pd.concat([pd.read_csv(i) for i in all_traces_summaries]).reset_index(drop=True)
df

,220,221,222,223,224,225,226,227,228,229,...,percent_dmso,well_volume,concentrations,Kpi /mM,Kcl/mM,shaking_seconds,BSA,bulk_dispensing,experiments,column
0,1.768,2.018,3.474,2.598,2.163,2.348,2.801,2.712,2.877,2.718,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.050,2.108,2.286,3.136,2.672,2.698,2.888,2.613,2.821,2.892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.986,2.156,2.665,2.758,2.092,2.319,2.543,2.630,2.809,2.725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.476,2.908,2.548,2.767,2.409,2.532,2.926,2.864,3.415,3.472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.014,3.480,3.480,3.017,2.454,2.746,3.480,3.480,3.480,3.348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13195,2.211,2.366,2.422,2.159,2.193,2.359,2.651,3.478,2.860,2.968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13196,2.190,3.049,2.650,2.433,2.372,3.235,2.877,2.839,3.480,3.119,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13197,2.247,2.470,3.483,2.897,2.539,3.483,3.320,3.248,3.483,3.483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13198,2.384,2.677,3.478,3.171,3.165,2.887,3.079,3.478,2.653,2.842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df[[i for i in df.columns if not i.isnumeric()]].T

,0,1,2,3,4,5,6,7,8,9,...,13190,13191,13192,13193,13194,13195,13196,13197,13198,13199
experiment_number,5,5,5,5,5,5,5,5,5,5,...,9,9,9,9,9,9,9,9,9,9
protein_concentration,4.01,4.01,4.01,4.01,4.01,4.01,4.01,4.01,4.01,4.01,...,15.14,15.14,15.14,15.14,15.14,15.14,15.14,15.14,15.14,15.14
plate_type,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,...,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660,Corning 3660
file,20190619_boi.CSV,20190619_boi.CSV,20190619_boi.CSV,20190619_boi.CSV,20190619_boi.CSV,20190619_boi.CSV,20190619_boi.CSV,20190619_boi.CSV,20190619_boi.CSV,20190619_boi.CSV,...,20190701.CSV,20190701.CSV,20190701.CSV,20190701.CSV,20190701.CSV,20190701.CSV,20190701.CSV,20190701.CSV,20190701.CSV,20190701.CSV
ligand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4-Phenylimidazole,4-Phenylimidazole,4-Phenylimidazole,4-Phenylimidazole,4-Phenylimidazole,4-Phenylimidazole,4-Phenylimidazole,4-Phenylimidazole,4-Phenylimidazole,4-Phenylimidazole
concentration,500.0,250.0,125.0,62.5,31.25,15.625,7.8125,0.0,500.0,250.0,...,7.8125,0.0,500.0,250.0,125.0,62.5,31.25,15.625,7.8125,0.0
control,False,False,False,False,False,False,False,False,True,True,...,False,False,True,True,True,True,True,True,True,True
address,A1,C1,E1,G1,I1,K1,M1,O1,B1,D1,...,M9,O9,B9,D9,F9,H9,J9,L9,N9,P9
experiment,5,5,5,5,5,5,5,5,5,5,...,9,9,9,9,9,9,9,9,9,9
block,1,1,1,1,1,1,1,1,1,1,...,9,9,9,9,9,9,9,9,9,9


In [6]:
print(f"{len(df['well_volume'].dropna()) / len(df):.2%}")

64.97%


In [5]:
df['control'].sum()

6600